In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import math

import pandas as pd
import numpy as np
import pennylane as qml
from pennylane import numpy as qnp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

First we load the datasets:

In [2]:
gluehwein = 'gluehweindorf'
krampus = 'krampuskogel'
lebkuchen = 'lebkuchenstadt'

villages = [gluehwein, krampus, lebkuchen]
datasets = {}

for village in villages:
        datasets[village] = pd.read_csv(f'{village}.csv')

There are no null values in the dataset and 500 entries

In [3]:
datasets[gluehwein].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [4]:
datasets[krampus].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [5]:
datasets[lebkuchen].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   carol_singing    500 non-null    float64
 1   snowball_energy  500 non-null    float64
 2   label            500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


Min-Max values

In [6]:
for name in (gluehwein, krampus, lebkuchen):
    print(f'{name}-carol_singing: min: {datasets[name]['carol_singing'].min()}, max: {datasets[name]['carol_singing'].max()}')
    print(f'{name}-snowball_energy: min: {datasets[name]['snowball_energy'].min()}, max: {datasets[name]['snowball_energy'].max()}')
    print()

gluehweindorf-carol_singing: min: -2.9804047381428838, max: 2.9371221242497527
gluehweindorf-snowball_energy: min: -2.9723258785728337, max: 3.141592653589793

krampuskogel-carol_singing: min: -3.141592653589793, max: 3.117552213512825
krampuskogel-snowball_energy: min: -2.8957902158960995, max: 2.8898416183940667

lebkuchenstadt-carol_singing: min: -1.620977694446723, max: 3.141592653589793
lebkuchenstadt-snowball_energy: min: -0.6755685974503365, max: 1.4214525718679278



Label values

In [7]:
for name in (gluehwein, krampus, lebkuchen):
    print(f'{name}-labels:  {datasets[gluehwein]['label'].unique()}')


gluehweindorf-labels:  [1 0]
krampuskogel-labels:  [1 0]
lebkuchenstadt-labels:  [1 0]


In [8]:
X_train = {}
y_train = {}

for name in (gluehwein, krampus, lebkuchen):
    X_train[name] = datasets[name][['carol_singing', 'snowball_energy']].values
    y_train[name] = datasets[name]['label'].values

# Encoding

## Amplitude encoding

In [9]:
def amplitude_encoding(inputs, wires):
    qml.AmplitudeEmbedding(
        features=inputs,
        wires=wires,
        normalize=True,
        pad_with=0.0
    )


## Angle encoding

In [10]:
def angle_encoding(inputs, wires):
    qml.AngleEmbedding(
        features=inputs, 
        wires=wires, 
        rotation='Y')

## Density matrix encoding


In [11]:
def density_matrix_encoding(inputs, wires):    
    x0, x1 = inputs
    rho = np.outer(inputs, inputs) / np.inner(inputs, inputs)

    qml.QubitDensityMatrix(rho, wires=wires)

# Entanglement

## Linear Entanglement

In [12]:
def linear_entanglement(wires):
    for i in range(len(wires) - 1):
        qml.CNOT(wires=[wires[i], wires[i + 1]])

## Circular Entanglement

In [13]:
def circular_entanglement(wires):
    linear_entanglement(wires)
    qml.CNOT(wires=[wires[-1], wires[0]])


## Full Entanglement

In [14]:
def full_entanglement(wires):
    for i in range(len(wires)):
        for j in range(i + 1, len(wires)):
            qml.CNOT(wires=[wires[i], wires[j]])


# QML Circuit

In [15]:
def make_qml_circuit(dev):
    
    @qml.qnode(dev)
    def qml_circuit(weights, inputs, encoding_fn, entanglement_fn, reuploading_count):
        encoding_fn(inputs, wires=list(range(num_qubits)))

        for j in range(reuploading_count):
            entanglement_fn(wires=list(range(num_qubits)))
            for i in range(num_qubits):
                qml.RY(weights[j][i], wires=i)

        return qml.expval(qml.PauliZ(0)) # this returns a number between the eigenvalues of Z, which are -1 and 1
    
    return qml_circuit

# Cost

In [16]:
def cost(weights, param):
    [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count] = param
    predictions = qnp.array([
        (qml_circuit(weights, x, encoding_fn, entanglement_fn, reuploading_count) * 0.5 + 1)
        for x in X
    ])
    return qnp.mean((predictions - Y) ** 2)

# Evaluation

## Gluehweindorf

In [17]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 2
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = amplitude_encoding
entanglement_fn = linear_entanglement 

X = X_train[gluehwein]
Y = y_train[gluehwein]


for reuploading_count in (1, 3):
    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)), 
        requires_grad=True
    )
    
    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]
        if epoch % 10 == 0 or epoch == 99: 
            print(f"Epoch {epoch + 1}: Cost = {current_cost:.4f} Weights = {weights}")
    

Epoch 1: Cost = 0.2500 Weights = [[3.13823507 0.91877414]]
Epoch 11: Cost = 0.2500 Weights = [[3.1386148  0.91877414]]
Epoch 21: Cost = 0.2500 Weights = [[3.14092443 0.91877414]]
Epoch 31: Cost = 0.2500 Weights = [[3.1451947  0.91877414]]
Epoch 41: Cost = 0.2500 Weights = [[3.14171614 0.91877414]]
Epoch 51: Cost = 0.2500 Weights = [[3.14008412 0.91877414]]
Epoch 61: Cost = 0.2500 Weights = [[3.14021617 0.91877414]]
Epoch 71: Cost = 0.2500 Weights = [[3.14089155 0.91877414]]
Epoch 81: Cost = 0.2500 Weights = [[3.1412679  0.91877414]]
Epoch 91: Cost = 0.2500 Weights = [[3.14142317 0.91877414]]
Epoch 100: Cost = 0.2500 Weights = [[3.14147944 0.91877414]]
Epoch 1: Cost = 0.6144 Weights = [[1.98426147 2.78941061]
 [0.08488078 2.16226564]
 [0.9643636  1.91529362]]
Epoch 11: Cost = 0.4651 Weights = [[ 2.13561616  2.90586482]
 [-0.03946384  2.01217803]
 [ 0.81319574  1.91529362]]
Epoch 21: Cost = 0.3478 Weights = [[2.29807148 2.85993045]
 [0.01384231 1.85785385]
 [0.65236557 1.91529362]]
Epoch

## Krampus

In [18]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 2
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = angle_encoding
entanglement_fn = circular_entanglement 

X = X_train[krampus]
Y = y_train[krampus]


for reuploading_count in (1, 3):
    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)), 
        requires_grad=True
    )
    
    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]
        if epoch % 10 == 0 or epoch == 99: 
            print(f"Epoch {epoch + 1}: Cost = {current_cost:.4f} Weights = {weights}")
    

Epoch 1: Cost = 0.4577 Weights = [[2.27239813 2.5172095 ]]
Epoch 11: Cost = 0.4498 Weights = [[2.37158926 2.5172095 ]]
Epoch 21: Cost = 0.4435 Weights = [[2.46713114 2.5172095 ]]
Epoch 31: Cost = 0.4387 Weights = [[2.55656896 2.5172095 ]]
Epoch 41: Cost = 0.4351 Weights = [[2.63863777 2.5172095 ]]
Epoch 51: Cost = 0.4323 Weights = [[2.7131636 2.5172095]]
Epoch 61: Cost = 0.4302 Weights = [[2.78064945 2.5172095 ]]
Epoch 71: Cost = 0.4286 Weights = [[2.84186859 2.5172095 ]]
Epoch 81: Cost = 0.4274 Weights = [[2.89760097 2.5172095 ]]
Epoch 91: Cost = 0.4265 Weights = [[2.94850665 2.5172095 ]]
Epoch 100: Cost = 0.4258 Weights = [[2.99061238 2.5172095 ]]
Epoch 1: Cost = 0.6953 Weights = [[0.44609773 0.35005176]
 [2.96703133 0.4990649 ]
 [0.3884364  2.82283908]]
Epoch 11: Cost = 0.6187 Weights = [[0.59328101 0.50242029]
 [2.88814448 0.65139856]
 [0.4569901  2.82283908]]
Epoch 21: Cost = 0.5361 Weights = [[0.75626224 0.66180965]
 [2.74904957 0.82085765]
 [0.5233128  2.82283908]]
Epoch 31: Cos

## Lebkuchen

In [19]:
opt = qml.AdamOptimizer(stepsize=0.01)

num_qubits = 4
dev = qml.device("default.qubit", wires=num_qubits)
qml_circuit = make_qml_circuit(dev)

encoding_fn = density_matrix_encoding
entanglement_fn = full_entanglement 

X = X_train[lebkuchen]
Y = y_train[lebkuchen]


for reuploading_count in (1, 3):
    weights = qnp.array(
        np.random.uniform(0, math.pi, size=(reuploading_count, num_qubits)), 
        requires_grad=True
    )
    
    param = [X, Y, qml_circuit, encoding_fn, entanglement_fn, reuploading_count]

    for epoch in range(100):
        args, current_cost = opt.step_and_cost(cost, weights, param)
        weights = args[0]
        if epoch % 10 == 0 or epoch == 99: 
            print(f"Epoch {epoch + 1}: Cost = {current_cost:.4f} Weights = {weights}")
    

DeviceError: Operator QubitDensityMatrix(array([[0.00170019, 0.04119832],
       [0.04119832, 0.99829981]]), wires=[0, 1, 2, 3]) not supported with default.qubit and does not provide a decomposition.